# Museum Analysis

This notebook connects to the museum database and conducts simple analysis of the data.

## Imports

In [60]:
from extract import get_config
from pipeline import get_connection, get_cursor

## Setup

In [61]:
config = get_config()
conn = get_connection(config)
cur = get_cursor(conn)

What exhibition is most frequently visited?

In [54]:
sql_query = """
            SELECT e.exhibition_name, COUNT(*)
            FROM (SELECT * FROM rating UNION SELECT * FROM special_requests) AS review
            LEFT JOIN exhibitions as e ON e.exhibition_id = review.exhibition_id
            GROUP BY exhibition_name;
            """

cur.execute(sql_query)
result = cur.fetchall()
print(result)

[['Thunder Lizards', 344], ['Adaptation', 705], ['Cetacean Sensations', 800], ['The Crenshaw Collection', 633], ['Our Polluted World', 793], ['Measureless to Man', 500]]


By assuming that the number of buttons pressed at the kiosk are by different people,
we querying the data from rating and special_requests table, and joining to the exhibition table for the names. The result shows us that can see that Cetacean Sensations has the most amount of data collected from the kiosks so we assume the most frequently visited exhibition is Cetacean Sensations.

What hour of the day has the most ratings?

In [55]:
sql_query = """
            SELECT 
            EXTRACT(HOUR from date) as hour,
            COUNT(*) as count
            FROM rating
            GROUP BY hour
            ORDER BY count DESC
            LIMIT 1; 

            """

cur.execute(sql_query)
result = cur.fetchall()
print(result)

[[Decimal('17'), 1238]]


The most ratings collected during 10am hour in a day.

What is the average rating for each exhibition?

In [56]:
sql_query = """
            SELECT e.exhibition_name, ROUND((AVG(score_id)::numeric),2)
            FROM (SELECT * FROM rating UNION SELECT * FROM special_requests) AS review
            LEFT JOIN exhibitions as e ON e.exhibition_id = review.exhibition_id
            GROUP BY exhibition_name;
            """
cur.execute(sql_query)
result = cur.fetchall()
print(result)

[['Thunder Lizards', Decimal('2.81')], ['Adaptation', Decimal('1.89')], ['Cetacean Sensations', Decimal('2.41')], ['The Crenshaw Collection', Decimal('1.45')], ['Our Polluted World', Decimal('1.16')], ['Measureless to Man', Decimal('1.86')]]


Rounded to 2 decimal place, the average ratings for each exhibitions are:
Adaptation: 1.89
Cetacean Sensations: 2.67
The Crenshaw Collection: 1.38
Our Polluted World: 1.19
Measureless to Man: 1.86

What proportion of all 4+ ratings are given to exhibition 4?

In [57]:
sub_query = """
            SELECT exhibition_id, COUNT(*) as total 
            FROM rating 
            WHERE score_id >= 4 
            GROUP BY exhibition_id"""

sql_query= f"""
           SELECT ROUND((exh_4_total/new_total)*100, 2)
           FROM
           (SELECT SUM(total) AS new_total,
           (SELECT SUM(total) FROM ({sub_query}) WHERE exhibition_id = 4) AS exh_4_total
           FROM
           ({sub_query}))
           ;

           """

cur.execute(sql_query)
result = cur.fetchall()
print(result)

[[Decimal('11.00')]]


12.89% of all 4+ ratings are given to exhibition 4.

Are positive ratings more frequent before or after 1pm?

In [58]:
sub_query = """
            SELECT EXTRACT(HOUR FROM date) as hour,
            count(*) as total
            FROM rating
            WHERE score_id >= 3
            GROUP BY EXTRACT(HOUR FROM date)
            ORDER BY hour ASC;
            """
query = """
        SELECT SUM(total) as after
        FROM
        (SELECT EXTRACT(HOUR FROM date) as hour,
        count(*) as total
        FROM rating
        WHERE score_id >= 3
        GROUP BY EXTRACT(HOUR FROM date))
        WHERE hour >= 13
        ;
        """
sql_query = """
            SELECT SUM(total) as before
            FROM
            (SELECT EXTRACT(HOUR FROM date) as hour,
            count(*) as total
            FROM rating
            WHERE score_id >= 3
            GROUP BY EXTRACT(HOUR FROM date))
            WHERE hour < 13
            ;"""
cur.execute(sub_query)
result = cur.fetchall()
print(result)

[[Decimal('0'), 2], [Decimal('2'), 2], [Decimal('3'), 1], [Decimal('4'), 1], [Decimal('5'), 1], [Decimal('6'), 3], [Decimal('8'), 35], [Decimal('9'), 69], [Decimal('10'), 74], [Decimal('11'), 72], [Decimal('12'), 63], [Decimal('13'), 70], [Decimal('14'), 60], [Decimal('15'), 54], [Decimal('16'), 258], [Decimal('17'), 429], [Decimal('18'), 22], [Decimal('20'), 2], [Decimal('21'), 2], [Decimal('23'), 2]]


In [62]:
cur.execute("SELECT * FROM rating ORDER BY rating_id DESC LIMIT 5;")
result = cur.fetchall()
print(result)

[[3642, 1, 4, datetime.datetime(2024, 5, 10, 6, 54, 43)], [3641, 1, 1, datetime.datetime(2024, 5, 10, 6, 42, 52)], [3640, 3, 3, datetime.datetime(2024, 5, 10, 6, 31, 58)], [3639, 3, 1, datetime.datetime(2024, 5, 10, 6, 20, 56)], [3638, 4, 5, datetime.datetime(2024, 5, 10, 6, 9, 54)]]
